# **Classification** des **séries temporelles** avec `sktime`

## Métadonnées

- **Expérience nº :** 5.1.3
- **Date :** 16/11/2023
- **Heure :** 15:50
- **Données :** `ready_Cloud9am_Cloud3pm.csv`
- **Tri :** index chronologique
- **Découpage :** `TimeSeriesSplit`
- **Mise à l'échelle :** `StandardScaler`
- **Rééquilibrage :** non
- **Conversion :** `numpyfy`
- **Algorithme :** KNN + DTW

## Sommaire

1. Initialisation
2. Découpage
3. Mise à l'échelle
4. Conversion
5. Modélisation
6. Évaluation

## 1. Initialisation

In [1]:
# Importation des bibliothèques et modules nécessaires au fonctionnement de ce notebook

import pandas as pd
import numpy as np

from sklearn.metrics import classification_report

In [2]:
# Importation du jeu de données et enregistrement dans le DataFrame `df`

df = pd.read_csv("../../../data/9df/Alex/ready_Cloud9am_Cloud3pm.csv", index_col = 1).sort_index()
df = df.drop(columns = "Unnamed: 0")

In [3]:
# Inspection de la structure de `df`

df.head()

,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustSpeed,WindSpeed9am,WindSpeed3pm,Humidity9am,Humidity3pm,...,month,day,LocationNum,WindGustDirNum,WindDir9amNum,WindDir3pmNum,Latitude,Longitude,CodeRegionNum,NonMesNum
Date,,,,,,,,,,,,,,,,,,,,,
2008-07-01,8.8,15.7,5.0,1.6,2.6,48.0,13.0,15.0,92.0,67.0,...,7,1,32,2.356194,3.926991,3.141593,-34.928181,138.599931,4,6
2008-07-02,12.7,15.8,0.8,1.4,7.8,35.0,13.0,15.0,75.0,52.0,...,7,2,32,3.926991,4.319690,3.926991,-34.928181,138.599931,4,6
2008-07-03,6.2,15.1,0.0,1.8,2.1,20.0,2.0,11.0,81.0,56.0,...,7,3,32,3.141593,1.178097,3.926991,-34.928181,138.599931,4,6
2008-07-04,5.3,15.9,0.0,1.4,8.0,30.0,6.0,13.0,71.0,46.0,...,7,4,32,1.178097,1.178097,0.785398,-34.928181,138.599931,4,6
2008-07-07,7.6,11.2,16.2,4.6,1.1,46.0,17.0,13.0,83.0,88.0,...,7,7,32,3.534292,2.748894,3.926991,-34.928181,138.599931,4,6


In [4]:
# Inspection de la structure de `df`

df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5959 entries, 2008-07-01 to 2017-06-25
Data columns (total 27 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   MinTemp         5959 non-null   float64
 1   MaxTemp         5959 non-null   float64
 2   Rainfall        5959 non-null   float64
 3   Evaporation     5959 non-null   float64
 4   Sunshine        5959 non-null   float64
 5   WindGustSpeed   5959 non-null   float64
 6   WindSpeed9am    5959 non-null   float64
 7   WindSpeed3pm    5959 non-null   float64
 8   Humidity9am     5959 non-null   float64
 9   Humidity3pm     5959 non-null   float64
 10  Pressure9am     5959 non-null   float64
 11  Pressure3pm     5959 non-null   float64
 12  Temp9am         5959 non-null   float64
 13  Temp3pm         5959 non-null   float64
 14  RainToday       5959 non-null   int64  
 15  RainTomorrow    5959 non-null   int64  
 16  year            5959 non-null   int64  
 17  month           5959 no

## 2. Découpage

In [5]:
# Découpage de `df` sur l'axe des colonnes : séparation des variables explicatives (`data`) et cible (`target`)

data = df.drop(columns = "RainTomorrow")
target = df["RainTomorrow"]

In [6]:
# Découpage de `data` et de `target` sur l'axe des lignes : séparation des jeux d'entraînement (`*_train`) et de test (`*_test`) avec le splitter `TimeSeriesSplit`

from sklearn.model_selection import TimeSeriesSplit

tss = TimeSeriesSplit(n_splits = 5) ## Nous fixons le paramètre `n_splits` à 5 afin d'avoir une répartition de 80 / 20 entre les jeux d'entraînement et de test, respectivement.

for train_index, test_index in tss.split(data):
    X_train, X_test = data.iloc[train_index, :], data.iloc[test_index,:]
    y_train, y_test = target.iloc[train_index], target.iloc[test_index]

## 3. Mise à l'échelle

In [7]:
# Mise à l'échelle obligatoire de `X_train` et de `X_test` pour assurer que le classificateur KNN fonctionne correctement

# Effectuée après la séparation des jeux d'entraînement et de test pour éviter que des informations concernant le second fuitent vers le premier

from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

## 4. Conversion

In [8]:
# Importation de la fonction artisanale `numpyfy`

from numpyfy import numpyfy

In [9]:
# Application de la fonction `numpyfy` aux 4 jeux de données issus du découpage effectué par le splitter `TimeSeriesSplit` afin de les convertir en arrays et ainsi les rendre compatibles avec `sktime`

X_train, X_test, y_train, y_test = numpyfy(X_train, X_test, y_train, y_test,
                                           X_to_numpy = False)

## 5. Modélisation

In [10]:
# Importation de la classe `KNeighborsTimeSeriesClassifier`

from sktime.classification.distance_based import KNeighborsTimeSeriesClassifier

In [11]:
# Instanciation d'un modèle classificateur avec `n_neighbors = 1` et `distance = "dtw"`

clf_knn_ts = KNeighborsTimeSeriesClassifier(n_neighbors = 1, distance = "dtw")

In [12]:
# Récupération des paramètres initiaux

clf_knn_ts.get_params()

{'algorithm': 'brute',
 'distance': 'dtw',
 'distance_mtype': None,
 'distance_params': None,
 'leaf_size': 30,
 'n_jobs': None,
 'n_neighbors': 1,
 'pass_train_distances': False,
 'weights': 'uniform'}

In [13]:
# Entraînement du modèle

clf_knn_ts.fit(X_train, y_train)

KNeighborsTimeSeriesClassifier()

In [14]:
# Récupération des paramètres ajustés

clf_knn_ts.get_fitted_params()

{'classes': array([0, 1]),
 'fit_time': 49,
 'knn_estimator': KNeighborsClassifier(algorithm='brute', metric='precomputed', n_neighbors=1),
 'n_classes': 2,
 'knn_estimator__classes': array([0, 1]),
 'knn_estimator__effective_metric': 'precomputed',
 'knn_estimator__effective_metric_params': {},
 'knn_estimator__n_features_in': 4966,
 'knn_estimator__n_samples_fit': 4966,
 'knn_estimator__outputs_2d': False}

In [15]:
# Réalisation des prédictions

y_pred = clf_knn_ts.predict(X_test)

## 6. Évaluation

In [16]:
# Élaboration de la matrice de confusion

pd.crosstab(y_test, y_pred, rownames = ["Classe réelle"], colnames = ["Classe prédite"])

Classe prédite,0,1
Classe réelle,,
0,607,118
1,103,165


In [17]:
# Élaboration du rapport de classification

print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.85      0.84      0.85       725
           1       0.58      0.62      0.60       268

    accuracy                           0.78       993
   macro avg       0.72      0.73      0.72       993
weighted avg       0.78      0.78      0.78       993

